In [1]:
import IPython
import numpy as np
from scipy.sparse.linalg import gmres

In [2]:
def gen_matrix(n: int, epsilon: np.float64) -> np.ndarray:
    """Generate the matrix A and b of the linear system Au = b."""
    size = n - 1
    A = np.zeros((size, size))
    b = np.ones((size, 1))
    for i in range(size):
        A[i, i] = 2 * epsilon * (n**2) + n
        if i > 0:
            A[i, i - 1] = -epsilon * (n**2) - n
        if i < size - 1:
            A[i, i + 1] = -epsilon * (n**2)
    return A, b

In [3]:
def u(x: np.float64 | np.ndarray, epsilon: np.float64) -> np.ndarray:
    """The original function $u(x)$ of Exercise 06 Problem 3."""

    def temp_func(x: np.float64 | np.ndarray) -> np.float64 | np.ndarray:
        return np.exp(-x / epsilon)

    temp = temp_func(np.float64(1))

    return np.where(
        np.logical_or(x == 0, x == 1),
        np.float64(0),
        x - (temp_func(1 - x) - temp) / (1 - temp),
    )

In [4]:
def gen_real_val(n: int, epsilon: np.float64) -> np.ndarray:
    """Generate the real values of the function u(x) at the grid points."""
    x = np.linspace(0, 1, n + 1)
    return u(x, epsilon)[1:-1].reshape(-1, 1)

In [5]:
def jacobi_method(
    A: np.ndarray, p: np.ndarray, tol: np.float64, max_iter: int
) -> tuple[np.ndarray, int]:
    """Solve the linear system of equations Au=p using the Jacobi method."""
    D = np.diag(np.diag(A))
    inv_D = np.linalg.inv(D)
    x = np.zeros_like(p)

    for i in range(max_iter):
        x = np.dot(inv_D, p - np.dot(A, x)) + x
        err = np.linalg.norm(A @ x - p)
        if err < tol:
            return x, i + 1
    return x, max_iter

In [6]:
def gauss_seidel_method(
    A: np.ndarray, p: np.ndarray, tol: np.float64, max_iter: int
) -> tuple[np.ndarray, int]:
    """Solve the linear system of equations Au=p using the Gauss Seidel method."""
    D = np.diag(np.diag(A))
    L = np.tril(A, -1)
    inv_D_plus_L = np.linalg.inv(D + L)
    x = np.zeros_like(p)

    for i in range(max_iter):
        x = np.dot(inv_D_plus_L, p - np.dot(A, x)) + x
        err = np.linalg.norm(A @ x - p)
        if err < tol:
            return x, i + 1
    return x, max_iter

In [7]:
def GMRES_method(
    A: np.ndarray,
    p: np.ndarray,
    tol: np.float64,
    max_iter: int,
    restart: int | None = None,
) -> tuple[np.ndarray, int]:
    """Solve the linear system of equations Au=p using the GMRES method."""
    iter_count = 1

    def callback(_: np.ndarray) -> None:
        nonlocal iter_count
        iter_count += 1

    # In GMRES method, restart is set to min(p.size(), restart) or min(p.size(), 20)
    # so we set restart to a large number.
    if restart is None:
        restart = 1000

    result, info = gmres(
        A,
        p,
        rtol=0,
        atol=float(tol),
        restart=restart,
        maxiter=max_iter,
        callback=callback,
        callback_type="x",
    )

    if info != 0:
        return result, info
    return result, iter_count

In [8]:
max_iter = 10000
tol = np.float64(1e-10)

n = 256

restarts = [5, 10, 20, 30, 40, 50]

epsilons = np.array([10**-i for i in [0, 2, 4, 6]])

In [9]:
for epsilon in epsilons:
    x_real = gen_real_val(n, epsilon)
    A, p = gen_matrix(n, epsilon)
    i = jacobi_method(A, p, tol, max_iter)
    display(f"Jacobi method: epsilon = {epsilon}, iterations = {i[1]}")

'Jacobi method: epsilon = 1.0, iterations = 10000'

'Jacobi method: epsilon = 0.01, iterations = 3956'

'Jacobi method: epsilon = 0.0001, iterations = 316'

'Jacobi method: epsilon = 1e-06, iterations = 267'

In [10]:
for epsilon in epsilons:
    x_real = gen_real_val(n, epsilon)
    A, p = gen_matrix(n, epsilon)
    i = gauss_seidel_method(A, p, tol, max_iter)
    IPython.display.display(
        f"Gauss Seidel method: epsilon = {epsilon}, iterations = {i[1]}"
    )

'Gauss Seidel method: epsilon = 1.0, iterations = 10000'

'Gauss Seidel method: epsilon = 0.01, iterations = 1856'

'Gauss Seidel method: epsilon = 0.0001, iterations = 32'

'Gauss Seidel method: epsilon = 1e-06, iterations = 7'

In [11]:
for epsilon in epsilons:
    x_real = gen_real_val(n, epsilon)
    A, p = gen_matrix(n, epsilon)
    i = GMRES_method(A, p, tol, max_iter)
    IPython.display.display(f"GMRES method: epsilon = {epsilon}, iterations = {i[1]}")

'GMRES method: epsilon = 1.0, iterations = 3'

'GMRES method: epsilon = 0.01, iterations = 2'

'GMRES method: epsilon = 0.0001, iterations = 2'

'GMRES method: epsilon = 1e-06, iterations = 2'

In [12]:
for restart in restarts:
    for epsilon in epsilons:
        x_real = gen_real_val(n, epsilon)
        A, p = gen_matrix(n, epsilon)
        i = GMRES_method(A, p, tol, max_iter, restart=restart)
        IPython.display.display(
            f"GMRES method with restart = {restart}: "
            f"epsilon = {epsilon}, iterations = {i[1]}"
        )

'GMRES method with restart = 5: epsilon = 1.0, iterations = 10000'

'GMRES method with restart = 5: epsilon = 0.01, iterations = 125'

'GMRES method with restart = 5: epsilon = 0.0001, iterations = 63'

'GMRES method with restart = 5: epsilon = 1e-06, iterations = 58'

'GMRES method with restart = 10: epsilon = 1.0, iterations = 3093'

...

'GMRES method with restart = 40: epsilon = 1e-06, iterations = 11'

'GMRES method with restart = 50: epsilon = 1.0, iterations = 114'

'GMRES method with restart = 50: epsilon = 0.01, iterations = 19'

'GMRES method with restart = 50: epsilon = 0.0001, iterations = 16'

'GMRES method with restart = 50: epsilon = 1e-06, iterations = 10'